# Face detection, tracking and matching:

## Import dependencies:

In [1]:
import face_recognition
import cv2
import os
from os.path import basename
import skvideo.io
import glob
import sys
import types
import subprocess
from random import randint
import numpy as np
import scipy.misc
from skimage.transform import rescale, resize, downscale_local_mean
import json
import keras
import gc
from keras.preprocessing import image
from keras.models import model_from_json
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.applications.inception_v3 import preprocess_input

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## OpenCV version check:

In [2]:
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

if int(major_ver)  < 3 :
    print ("Update OpenCV ...")
    sys.exit(1)

## Load video files:

In [3]:
source = './Drag_Me_Down.mp4'
try:
    video_capture = cv2.VideoCapture(source)
    print ("Imported video using Open-CV ...")
except:
    video_capture =  skvideo.io.vread(source)
    print ("Imported video using sci-kit video ...")

Imported video using Open-CV ...


## Initialize variables for video processing:

In [4]:
sgd = SGD(lr=1e-7, decay=0.5, momentum=1, nesterov=True)
rms = RMSprop(lr=1e-7, rho=0.9, epsilon=1e-08, decay=0.0)
ada = Adagrad(lr=1e-7, epsilon=1e-08, decay=0.0)
optimizer = sgd
IMG_HEIGHT = 299
IMG_WIDTH = 299

In [5]:
length = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

save_path = "./proc_vid.mp4"
save_audio = "./audio.wav"
save_path_w_audio = "./proc_vid_audio.mp4"
output_dir = './output/'

In [6]:
face_locations = []
face_encodings = []
face_names = []
frame_number = 0
face_count = 0
process_this_frame = True
inverse_scale_factor = 2

w, h = int(video_capture.get(3)),int(video_capture.get(4))

print ("Source image width: "+ str(w))
print ("Source image height: "+ str(h))

fps = video_capture.get(cv2.CAP_PROP_FPS)
print ("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))

Source image width: 1280
Source image height: 720
Frames per second using video.get(cv2.CAP_PROP_FPS) : 23.976023976023978


In [7]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_writer = cv2.VideoWriter(save_path, fourcc, fps, (w,h), True)

In [8]:
reference_image_path = "./ref_img/"
file_list = glob.glob(reference_image_path + '/*.jpg')

In [9]:
n_proc_frames = length
resize_img = False
verbose = True
gen_train_img = True
interleaved = False
use_deep_learning = True
annotate = True

In [10]:
def compile_model(model):
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
def load_prediction_model(args):
    try:
        with open(args.config_file[0]) as json_file:
              model_json = json_file.read()
        model = model_from_json(model_json)
    except:
          print ("Please specify a model configuration file ...")
          sys.exit(1)
    try:
          model.load_weights(args.weights_file[0])
          print ("Loaded model weights from: " + str(args.weights_file[0]))
    except:
          print ("Error loading model weights ...")
          sys.exit(1)
    try:
        with open(args.labels_file[0]) as json_file:
            labels = json.load(json_file)
        print ("Loaded labels from: " + str(args.labels_file[0]))
    except:
        print ("No labels loaded ...")
        sys.exit(1)
    return model, labels

In [12]:
def gen_predict(model):
    try:
        compile_model(model)
        print ("Model successfully compiled ...")
    except:
        print ("Model failed to compile ...")

    print ("Compiling predictor function ...")                                          # to avoid the delay during video capture.
    _ = model.predict(np.zeros((1, n, n, 3), dtype=np.float32), batch_size=1)
    print ("Compilation completed ...")

In [13]:
args = types.SimpleNamespace()
args.config_file = ['./model/trained_2018_02_27-09_12_26_config_ft_.json']
args.weights_file = ['./model/trained_2018_02_27-09_12_26_weights_ft_.model']
args.labels_file = ['./model/trained_labels.json']
args.output_dir = ['./output/']

In [14]:
model, labels = load_prediction_model(args)

Loaded model weights from: ./model/trained_2018_02_27-09_12_26_weights_ft_.model
Loaded labels from: ./model/trained_labels.json


In [ ]:
while (video_capture.isOpened()):    
    ret, frame = video_capture.read() # Grab a single frame of video
    
    frame_number += 1
    
    if resize_img ==True:
        isf = inverse_scale_factor
        small_frame = cv2.resize(frame, (0, 0), fx=(1/isf), fy=(1/isf)) # Resize frame of video to 1/inverse_scale_factor size for faster processing
    else:
        isf = 1
        small_frame = frame
    if frame_number <=n_proc_frames:
        if ret ==True:
            if process_this_frame:
                face_locations = face_recognition.face_locations(small_frame) # Find all the faces and face encodings in the current frame of video
                face_encodings = face_recognition.face_encodings(small_frame, face_locations)
                face_names = []
                if annotate == True:
                    for face_encoding in face_encodings:
                        for file_path in file_list:
                            reference_image = face_recognition.load_image_file(file_path)
                            try:
                                reference_face_encoding = face_recognition.face_encodings(reference_image)[0]
                                if verbose == True:
                                    print ("Processed face encodings ...")
                                else:
                                    pass
                            except:
                                if verbose == True:
                                    print("Failed processing face encodings ...")
                                else:
                                    pass
                            name_ID = (os.path.splitext(basename(file_path))[0])
                            name_ID = name_ID.replace("_", " ")
                            match = face_recognition.compare_faces([reference_face_encoding], face_encoding) # See if the face is a match for the known face(s)
                            name = "Unknown"
                            if match[0]:
                                name = name_ID
                            face_names.append(name)
                else:
                    if verbose == True:
                        print ("Skipping face recognition mode ...")
                    else:
                        pass
            else:
                if verbose == True:
                    print ("Skipping frame ...")
                else:
                    pass
            if interleaved == True:
                process_this_frame = not process_this_frame # Only process every other frame of video to save time
            else:
                process_this_frame = process_this_frame

            # Display the results
            for (top, right, bottom, left), name in zip(face_locations, face_names):
                # Scale back up face locations since the frame we detected in was scaled to scaling factor size
                top *= int(isf)
                right *= int(isf)
                bottom *= int(isf)
                left *= int(isf)
                # Draw an ellipse around the face
                ex = left
                ey = top
                ew = int(abs(right - ex))
                eh = int(abs(bottom - ey))
                p1 = int(ew/2 + ex)
                p2 = int(eh/2 + ey)
                h1 = int(ew/2)
                h2 = int(eh/2)
                square = frame[max((ey-eh//2,0)):ey+3*eh//2, max((ex-ew//2,0)):ex+3*ew//2]
                if gen_train_img == True:
                    random_number = randint(10000000, 99999999)
                    random_number = str(random_number)
                    cv2.imwrite(os.path.join(output_dir+"//"+str(random_number)+"frame_%d.jpg" % face_count), square)
                    if verbose == True:
                        print ("Saved frame: "+ str(face_count)+" with face detected ..." )
                    else:
                        pass
                    face_count += 1
                else:
                    pass
                cv2.ellipse(frame, (p1, p2), (h1,h2), 0,0,360, (0,255,0), 2)
                if use_deep_learning == True and annotate == True:
                    square = cv2.resize(square.astype(np.float32),    \
                                        dsize=(IMG_WIDTH, IMG_HEIGHT),\
                                        interpolation = cv2.INTER_CUBIC)
                    try:
                        _X_ = image.img_to_array(square)
                        del (square)
                        _X_ = np.expand_dims(_X_, axis=0)
                        _X_ = preprocess_input(_X_)
                        probabilities = model.predict(_X_, batch_size=1).flatten()
                        del (_X_)
                        prediction = labels[np.argmax(probabilities)]
                        name = (str(prediction)).replace("_", " ")
                        print ("Face recognition using deep-learning ...")
                        print (prediction + "\t" + "\t".join(map(lambda x: "%.2f" % x, probabilities)))
                        print (str(prediction))
                        del (prediction)
                    except:
                        print ("Failed to create a prediction ...")         
                else:
                    pass
                if annotate == True:
                    font = cv2.FONT_HERSHEY_DUPLEX
                    cv2.rectangle(frame, (p1 - 100, bottom - 2), (p1 + 100, bottom + 33), (0, 0, 255), cv2.FILLED) 
                    cv2.putText(frame, name, (p1  - 94, bottom + 23 ), font, 0.75, (255, 255, 255), 1) # Draw a label with a name below the face
                else:
                    if verbose == True:
                        print ("No identifiers to annotate. Try setting annotate flag to True ...")
                    else:
                        pass
            try:
                video_writer.write(frame)
                if verbose == True:
                    print("Processed frame {} / {}".format(frame_number, length))
                else:
                    pass
            except:
                if verbose == True:
                    print("Failed writing frame {} / {}".format(frame_number, length))
                else:
                    pass
        else:
            if verbose == True:
                print("No frame to process ...")
            else:
                pass
    else:
        if verbose == True:
            print ("Processed "+ str(n_proc_frames) + " frames")
            print ("Detected " + str(face_count) + " faces" )
        else:
            print ("Detected " + str(face_count) + " faces" )
        break

Processed frame 1 / 4597
Processed frame 2 / 4597
Processed frame 3 / 4597
Processed frame 4 / 4597
Processed frame 5 / 4597
Processed frame 6 / 4597
Processed frame 7 / 4597
Processed frame 8 / 4597
Processed frame 9 / 4597
Processed frame 10 / 4597
Processed frame 11 / 4597
Processed frame 12 / 4597
Processed frame 13 / 4597
Processed frame 14 / 4597
Processed frame 15 / 4597
Processed frame 16 / 4597
Processed frame 17 / 4597
Processed frame 18 / 4597
Processed frame 19 / 4597
Processed frame 20 / 4597
Processed frame 21 / 4597
Processed frame 22 / 4597
Processed frame 23 / 4597
Processed frame 24 / 4597
Processed frame 25 / 4597
Processed frame 26 / 4597
Processed frame 27 / 4597
Processed frame 28 / 4597
Processed face encodings ...
Saved frame: 0 with face detected ...
Face recognition using deep-learning ...
Unknown	0.02	0.09	0.10	0.06	0.73
Unknown
Processed frame 29 / 4597
Processed face encodings ...
Saved frame: 1 with face detected ...
Face recognition using deep-learning ..

Face recognition using deep-learning ...
Liam_Payne	0.01	0.64	0.29	0.02	0.04
Liam_Payne
Saved frame: 47 with face detected ...
Face recognition using deep-learning ...
Unknown	0.03	0.11	0.19	0.03	0.64
Unknown
Processed frame 46 / 4597
Processed face encodings ...
Processed face encodings ...
Processed face encodings ...
Saved frame: 48 with face detected ...
Face recognition using deep-learning ...
Unknown	0.08	0.10	0.07	0.18	0.58
Unknown
Saved frame: 49 with face detected ...
Face recognition using deep-learning ...
Liam_Payne	0.01	0.57	0.28	0.02	0.12
Liam_Payne
Saved frame: 50 with face detected ...
Face recognition using deep-learning ...
Louis_Tomlinson	0.01	0.21	0.46	0.04	0.29
Louis_Tomlinson
Processed frame 47 / 4597
Processed face encodings ...
Processed face encodings ...
Processed face encodings ...
Saved frame: 51 with face detected ...
Face recognition using deep-learning ...
Unknown	0.05	0.08	0.10	0.36	0.41
Unknown
Saved frame: 52 with face detected ...
Face recognition usi

Face recognition using deep-learning ...
Unknown	0.01	0.03	0.02	0.00	0.93
Unknown
Processed frame 62 / 4597
Processed face encodings ...
Processed face encodings ...
Processed face encodings ...
Processed face encodings ...
Saved frame: 99 with face detected ...
Face recognition using deep-learning ...
Unknown	0.02	0.34	0.05	0.01	0.58
Unknown
Saved frame: 100 with face detected ...
Face recognition using deep-learning ...
Unknown	0.02	0.05	0.20	0.11	0.62
Unknown
Saved frame: 101 with face detected ...
Face recognition using deep-learning ...
Louis_Tomlinson	0.02	0.08	0.70	0.02	0.18
Louis_Tomlinson
Saved frame: 102 with face detected ...
Face recognition using deep-learning ...
Unknown	0.01	0.02	0.01	0.00	0.97
Unknown
Processed frame 63 / 4597
Processed face encodings ...
Processed face encodings ...
Processed face encodings ...
Processed face encodings ...
Saved frame: 103 with face detected ...
Face recognition using deep-learning ...
Unknown	0.03	0.05	0.16	0.11	0.64
Unknown
Saved fra

Processed face encodings ...
Processed face encodings ...
Processed face encodings ...
Saved frame: 149 with face detected ...
Face recognition using deep-learning ...
Niall_Horan	0.03	0.09	0.15	0.45	0.29
Niall_Horan
Saved frame: 150 with face detected ...
Face recognition using deep-learning ...
Unknown	0.01	0.03	0.01	0.06	0.89
Unknown
Saved frame: 151 with face detected ...
Face recognition using deep-learning ...
Unknown	0.01	0.18	0.34	0.03	0.44
Unknown
Processed frame 79 / 4597
Processed face encodings ...
Processed face encodings ...
Processed face encodings ...
Saved frame: 152 with face detected ...
Face recognition using deep-learning ...
Niall_Horan	0.04	0.08	0.13	0.54	0.20
Niall_Horan
Saved frame: 153 with face detected ...
Face recognition using deep-learning ...
Unknown	0.04	0.09	0.08	0.35	0.44
Unknown
Saved frame: 154 with face detected ...
Face recognition using deep-learning ...
Unknown	0.01	0.14	0.34	0.03	0.47
Unknown
Processed frame 80 / 4597
Processed face encodings .

Face recognition using deep-learning ...
Unknown	0.02	0.07	0.10	0.03	0.78
Unknown
Processed frame 92 / 4597
Processed face encodings ...
Processed face encodings ...
Processed face encodings ...
Processed face encodings ...
Saved frame: 200 with face detected ...
Face recognition using deep-learning ...
Liam_Payne	0.02	0.70	0.07	0.08	0.13
Liam_Payne
Saved frame: 201 with face detected ...
Face recognition using deep-learning ...
Unknown	0.02	0.10	0.28	0.04	0.56
Unknown
Saved frame: 202 with face detected ...
Face recognition using deep-learning ...
Unknown	0.07	0.08	0.27	0.16	0.42
Unknown
Saved frame: 203 with face detected ...
Face recognition using deep-learning ...
Unknown	0.03	0.05	0.09	0.03	0.81
Unknown
Processed frame 93 / 4597
Processed face encodings ...
Processed face encodings ...
Processed face encodings ...
Processed face encodings ...
Saved frame: 204 with face detected ...
Face recognition using deep-learning ...
Louis_Tomlinson	0.01	0.11	0.51	0.03	0.34
Louis_Tomlinson
Sa

...

Processed face encodings ...
Saved frame: 4174 with face detected ...
Face recognition using deep-learning ...
Louis_Tomlinson	0.07	0.11	0.44	0.31	0.07
Louis_Tomlinson
Processed frame 4371 / 4597
Processed face encodings ...
Saved frame: 4175 with face detected ...
Face recognition using deep-learning ...
Unknown	0.15	0.12	0.15	0.27	0.31
Unknown
Processed frame 4372 / 4597
Processed frame 4373 / 4597
Processed frame 4374 / 4597
Processed frame 4375 / 4597
Processed face encodings ...
Saved frame: 4176 with face detected ...
Face recognition using deep-learning ...
Niall_Horan	0.03	0.03	0.11	0.76	0.07
Niall_Horan
Processed frame 4376 / 4597
Processed face encodings ...
Saved frame: 4177 with face detected ...
Face recognition using deep-learning ...
Niall_Horan	0.02	0.04	0.14	0.73	0.07
Niall_Horan
Processed frame 4377 / 4597
Processed face encodings ...
Saved frame: 4178 with face detected ...
Face recognition using deep-learning ...
Niall_Horan	0.02	0.10	0.29	0.42	0.17
Niall_Horan
Proc

Face recognition using deep-learning ...
Harry_Styles	0.66	0.02	0.10	0.15	0.06
Harry_Styles
Processed frame 4439 / 4597
Processed face encodings ...
Processed face encodings ...
Processed face encodings ...
Saved frame: 4214 with face detected ...
Face recognition using deep-learning ...
Unknown	0.02	0.38	0.18	0.01	0.41
Unknown
Saved frame: 4215 with face detected ...
Face recognition using deep-learning ...
Unknown	0.03	0.18	0.33	0.08	0.38
Unknown
Saved frame: 4216 with face detected ...
Face recognition using deep-learning ...
Harry_Styles	0.45	0.04	0.25	0.10	0.16
Harry_Styles
Processed frame 4440 / 4597
Processed face encodings ...
Processed face encodings ...
Processed face encodings ...
Saved frame: 4217 with face detected ...
Face recognition using deep-learning ...
Liam_Payne	0.01	0.49	0.36	0.03	0.12
Liam_Payne
Saved frame: 4218 with face detected ...
Face recognition using deep-learning ...
Louis_Tomlinson	0.03	0.11	0.47	0.13	0.26
Louis_Tomlinson
Saved frame: 4219 with face det

Face recognition using deep-learning ...
Unknown	0.04	0.29	0.23	0.06	0.38
Unknown
Saved frame: 4262 with face detected ...
Face recognition using deep-learning ...
Harry_Styles	0.61	0.04	0.18	0.10	0.07
Harry_Styles
Processed frame 4457 / 4597
Processed face encodings ...
Processed face encodings ...
Saved frame: 4263 with face detected ...
Face recognition using deep-learning ...
Harry_Styles	0.56	0.09	0.21	0.08	0.07
Harry_Styles
Saved frame: 4264 with face detected ...
Face recognition using deep-learning ...
Harry_Styles	0.31	0.23	0.12	0.08	0.27
Harry_Styles
Processed frame 4458 / 4597
Processed face encodings ...
Processed face encodings ...
Saved frame: 4265 with face detected ...
Face recognition using deep-learning ...
Unknown	0.07	0.21	0.21	0.04	0.47
Unknown
Saved frame: 4266 with face detected ...
Face recognition using deep-learning ...
Harry_Styles	0.42	0.15	0.24	0.05	0.15
Harry_Styles
Processed frame 4459 / 4597
Processed face encodings ...
Processed face encodings ...
Saved

Face recognition using deep-learning ...
Unknown	0.01	0.24	0.08	0.01	0.66
Unknown
Processed frame 4477 / 4597
Processed face encodings ...
Processed face encodings ...
Saved frame: 4311 with face detected ...
Face recognition using deep-learning ...
Harry_Styles	0.83	0.02	0.09	0.03	0.03
Harry_Styles
Saved frame: 4312 with face detected ...
Face recognition using deep-learning ...
Liam_Payne	0.01	0.49	0.10	0.01	0.39
Liam_Payne
Processed frame 4478 / 4597
Processed face encodings ...
Processed face encodings ...
Saved frame: 4313 with face detected ...
Face recognition using deep-learning ...
Harry_Styles	0.91	0.01	0.03	0.01	0.03
Harry_Styles
Saved frame: 4314 with face detected ...
Face recognition using deep-learning ...
Unknown	0.00	0.39	0.14	0.01	0.46
Unknown
Processed frame 4479 / 4597
Processed face encodings ...
Processed face encodings ...
Saved frame: 4315 with face detected ...
Face recognition using deep-learning ...
Harry_Styles	0.94	0.01	0.02	0.01	0.02
Harry_Styles
Saved fra

Processed frame 4519 / 4597
Processed frame 4520 / 4597
Processed frame 4521 / 4597
Processed frame 4522 / 4597
Processed frame 4523 / 4597
Processed frame 4524 / 4597
Processed frame 4525 / 4597
Processed frame 4526 / 4597
Processed frame 4527 / 4597
Processed frame 4528 / 4597
Processed frame 4529 / 4597
Processed frame 4530 / 4597
Processed frame 4531 / 4597
Processed frame 4532 / 4597
Processed frame 4533 / 4597
Processed frame 4534 / 4597
Processed frame 4535 / 4597
Processed frame 4536 / 4597
Processed frame 4537 / 4597
Processed frame 4538 / 4597
Processed frame 4539 / 4597
Processed frame 4540 / 4597
Processed frame 4541 / 4597
Processed frame 4542 / 4597
Processed frame 4543 / 4597
Processed frame 4544 / 4597
Processed frame 4545 / 4597
Processed frame 4546 / 4597
Processed frame 4547 / 4597
Processed frame 4548 / 4597
Processed frame 4549 / 4597
Processed frame 4550 / 4597
Processed frame 4551 / 4597
Processed frame 4552 / 4597
Processed frame 4553 / 4597
Processed frame 4554

## Release handle reading the video file or webcam:

In [ ]:
video_capture.release()
video_writer.release()

## Extract audio from a video file:

In [ ]:
cmd = 'ffmpeg -i %s -ab 320000 -ac 2 -ar 44100 -vn %s' % (source, save_audio)
print (cmd)
subprocess.call(cmd, shell=True)

## Copy audio track from one video to another:

In [ ]:
cmd = 'ffmpeg -y -i %s -i %s -shortest -c:v copy -c:a aac -b:a 256k  %s' % (save_path, save_audio, save_path_w_audio)
print (cmd)
subprocess.call(cmd, shell=True)
print('Muxing completed ...')
print('Saved output file to: %s' % (save_path_w_audio))